This code must be run locally, never on Google Colab, because there are functions that don't work properly at this platform.

In [ ]:
# When running it, change the directoy to that where this file is in.
directory  = "/home/merlin/coding/ic/createDataset"

In [1]:
# Google Drive folder IDs:
dataWaiting = 
dataWithResults = 

In [ ]:
import os
import glob
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.webdriver import WebDriver

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pydrive.files import GoogleDriveFile

import pytesseract
import PIL.Image

In [ ]:
# Verifying Google Drive API authorization. For this is necessary a Google account. To avoid using personal accounts,
# there is a specific account to do this, that should be logged in, when asked, to script work properly.
# Email:
# Password:

gAuth = GoogleAuth()
gAuth.LocalWebserverAuth()
gDrive = GoogleDrive(gAuth)

In [ ]:
def openWebsite() -> WebDriver:
    '''
    Initializes and opens an Google Chrome tab on login page at Maricondi website.
    '''

    # Setting the desired configuration.
    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', {
        "download.default_directory": directory,
        "savefile.default_directory": directory,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "plugins.always_open_pdf_externally": True
    })
    driver = webdriver.Chrome(options = options)

    # Opening website.
    driver.get("https://pacientes.labmaricondi.com.br/wfrmLogin.aspx")

    return driver

def getLogin() -> tuple[str, str]:
    '''
    Uses Pytesseract to read the user and password information on "ft_guia.jpeg".
    '''

    # Getting all the text possible on "ft_guia.jpeg".
    myconfig = r"--psm 1 --oem 3"
    text = pytesseract.image_to_string(PIL.Image.open("ft_guia.jpeg"), config = myconfig)

    # Adjusting the start reading point.
    userOffset = text.find("USUARIO: ") + 9
    passwordOffset = text.find("SENHA: ") + 7

    # Getting the user and password data.
    user = ""
    password = ""
    while text[userOffset].isnumeric():
        user = user + text[userOffset]
        userOffset += 1
    while text[passwordOffset].isnumeric():
        password = password + text[passwordOffset]
        passwordOffset += 1

    os.remove(directory + "/ft_guia.jpeg")

    return (user, password)

def logIn(driver: WebDriver, user: str, password: str):
    '''
    Uses Selenium to log in to the Maricondi website.
    '''

    # Writing user and password information.
    driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div[1]/div/div[1]/div[1]/div/input").send_keys(user)
    driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div[1]/div/div[1]/div[2]/div/input").send_keys(password)

    # Clicking on "ENTRAR" button.
    driver.find_element(By.XPATH, "/html/body/form/div[3]/div/div[1]/div/div[1]/div[3]/button").click()

    # Waiting to load new page.
    time.sleep(3)

def downloadResult(driver: WebDriver):
    '''
    Uses Selenium to download the pacient (user) exam results PDF.
    '''

    # Clicking on the PDF download button and waiting it.
    driver.find_element(By.XPATH, "/html/body/div/app-root/div/mx-resultados/section[2]/div/div/div/div[1]/div/div[2]/div/button[1]").click()
    time.sleep(25)

def uploadResult(user: GoogleDriveFile, gDrive: GoogleDrive):
    '''
    Uploads the exam results PDF on the Google Drive pacient (user) folder.
    '''

    pdfFile = glob.glob("Laudo*")[0]
    with open(pdfFile, "r") as f:
        fileName = os.path.basename(f.name)
        filePath = os.path.join(directory, fileName)
        driveFile = gDrive.CreateFile({
            'parents': [{'id': user['id']}],
            'title': "laudo.pdf"
        })
        driveFile.SetContentFile(filePath)
        driveFile.Upload()
        os.remove(filePath)

def movePacientFolder(pacient: GoogleDriveFile):
    '''
    Transfers a pacient (user) folder from 'Dados com Resultado' to 'Dados no Dataset'.
    '''

    pacient['parents'] = [{'id': dataWithResults}]
    pacient.Upload()

In [ ]:
# Get a list of pacients that are waiting their exam results.
userList = gDrive.ListFile({'q':  "\'" + dataWaiting + "\' in parents and trashed=false" }).GetList()

# Iterating through the pacients.
for user in userList:
    # Opening Maricondi website.
    driver = openWebsite()

    # Getting a list of all files on the Google Drive pacient folder.
    fileList = gDrive.ListFile({'q':  "\'" + user['id'] + "\' in parents and trashed=false" }).GetList()

    # Iterating through the files.
    for file in fileList:

        # When find the "ft_guia.jpeg" file...
        if file['title'] == "ft_guia.jpeg":
            file.GetContentFile(file['title'])

            try:
                login = getLogin() # Reading "ft_guia.jpeg" and getting user and password.
                logIn(driver, login[0], login[1]) # Using them to log in to the website.
                downloadResult(driver) # Downloading the exam result PDF.
                uploadResult(user, gDrive) # Uploading PDF on Google Drive.
                movePacientFolder(user) # Moving pacient folder (Dados em Espera do Resultado -> Dados com Resultado).

            # If something wents wrong, printing the pacient ID. The major cause of error is the OCR of "ft_guia.jpeg"
            # extracting invalid user and/or password. In this cases, we need to extract them by ourselves.
            except:
                print("Erro no usuário: ", user['title'])

            break

    # Closing the open Chrome tab.
    driver.close()